In [ ]:
# import packages
import numpy as np
import pandas as pd
import os
import datetime as dt
import stat
from scipy.stats import pointbiserialr, pearsonr
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns

#for text processing
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
nltk.download('wordnet')
import string
import gensim
import sklearn
from sklearn.preprocessing import MinMaxScaler
!pip install pickle5
import pickle5 as pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
pd.set_option('display.max_colwidth', 100)

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
id = '1Hqe-jJ-97wuX1PpI9dTKW16bsE2yh-Oi'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('final_test_data')  
# Dataset is now stored in a Pandas Dataframe

#pip install --upgrade pandas==1.3.4

with open("final_test_data", "rb") as fh:
  test_data = pickle.load(fh)


In [ ]:
test_data = test_data.rename(columns={'dirty_cosine_similarity':'cosine_similarity_dirty_comments', 'clean_cosine_similarity':'cosine_similarity_clean_comments', 'sentiment':'comment_sentiment'})

In [ ]:
test_data.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment,lemmatized_comment,lemmatized_parent_comment,clean_comment,clean_parent_comment,lemmatized_clean_comment,lemmatized_clean_parent_comment,vectorized_comment,vectorized_clean_comment,vectorized_parent_comment,vectorized_clean_parent_comment,cosine_similarity_dirty_comments,cosine_similarity_clean_comments,word_count,punctuation_count,has_repeated,exclaim_count,qns_mark_count,ellipses_mark_count,interjection_count,laughter_words_count,capitalized_word_count,partial_capital_word_count,emoticon_count,comment_sentiment
794682,1,Racist comments are really rare on the internet so it's clearly the readers bias and not the rat...,Targash,news,6.0,6.0,0.0,2014-06,11/6/14 16:02,"perhaps it is your bias dear redditor, and none of my own.",Racist comment really rare internet 's clearly reader bias rate occurrence one encounter racism ...,"perhaps bias dear redditor , none .",Racist comments are really rare on the internet so its clearly the readers bias and not the rate...,perhaps it is your bias dear redditor and none of my own,Racist comment really rare internet clearly reader bias rate occurrence one encounter racism onl...,perhaps bias dear redditor none,"[-0.2260461, -0.5411476, -0.5956079, 1.6732603, -0.35423955, -0.28117514, 1.0684463, 0.13423854,...","[-0.5515428, -0.59989977, 0.069325216, 1.5927559, -0.52806437, -0.07231382, 0.7368921, 0.7780658...","[-0.24906166, -0.65218884, 0.12795807, 0.646011, -0.11237952, 1.1585653, -0.18005028, -0.1678803...","[-0.33440658, -0.508268, 0.3577558, 0.4706608, 0.048984274, 1.179587, -0.04562621, -0.43975627, ...",0.250085,0.188576,27,2,0,0,0,0,0,0,0,0,0,neg
1003517,0,"yes, if good taste is what a cock doodle should do, then let a cock a doodle do.",superfusion1,sex,8.0,8.0,0.0,2011-01,19/1/11 15:32,That's the most tasteful video I have ever seen of a cock doodle.,"yes , good taste cock doodle , let cock doodle .",That 's tasteful video I ever see cock doodle .,yes if good taste is what a cock doodle should do then let a cock a doodle do,Thats the most tasteful video I have ever seen of a cock doodle,yes good taste cock doodle let cock doodle,Thats tasteful video I ever see cock doodle,"[-0.7737616, 0.4774656, 0.13742262, 0.18524809, -0.7151735, -0.11188578, 0.75100416, 0.35352728,...","[-0.76534796, 0.46344534, -0.03482504, -0.16052239, -0.7406304, 0.19035932, 0.670916, 0.35256463...","[-0.19669841, 0.023759527, 0.39963058, -0.104647115, -0.15868066, -0.20109259, -0.1438785, -0.20...","[-0.32654247, 0.2684772, 0.13918118, 0.002365145, -0.12061546, -0.17676899, -0.4078152, -0.27219...",0.249204,0.183911,18,3,0,0,0,0,0,0,0,0,0,pos
526207,1,Totally not photoshopped...,RB30DETT,funny,11.0,11.0,0.0,2015-12,11/12/15 17:29,London Underground,Totally photoshopped ...,London Underground,Totally not photoshopped,London Underground,Totally photoshopped,London Underground,"[-0.14899243, -0.19366069, -0.20084605, 0.6875266, -0.20740944, 0.04641313, -0.015980119, -0.109...","[-0.11780262, -0.2061242, 0.010147771, 0.5353635, -0.105101734, 0.22466993, -0.017185103, -0.176...","[0.20910913, -0.055749796, 0.017575901, 0.48273495, -0.31094885, -0.11349381, 0.12859, 0.0609374...","[0.11732921, -0.14933924, 0.042169612, 0.49933022, -0.33947077, -0.084573105, 0.21897213, 0.0413...",0.760832,0.762000,3,3,0,0,0,1,0,0,0,0,0,neu
169751,1,"No, there's too much sexism in those industries and that makes it dangerous for women to join tr...",randomcombination102,TumblrInAction,50.0,-1.0,-1.0,2016-12,18/12/16 2:17,"Yeah, it's much easier to bitch about male privilege on the internet than learn a skilled trade","No , 's much sexism industry make dangerous woman join trade .","Yeah , 's much easy bitch male privilege internet learn skilled trade",No theres too much sexism in those industries and that makes it dangerous for women to join trades,Yeah its much easier to bitch about male privilege on the internet than learn a skilled trade,No there much

In [ ]:
test_data.shape

(202166, 34)

In [ ]:
test_data.columns

Index(['label', 'comment', 'author', 'subreddit', 'score', 'ups', 'downs',
       'date', 'created_utc', 'parent_comment', 'lemmatized_comment',
       'lemmatized_parent_comment', 'clean_comment', 'clean_parent_comment',
       'lemmatized_clean_comment', 'lemmatized_clean_parent_comment',
       'vectorized_comment', 'vectorized_clean_comment',
       'vectorized_parent_comment', 'vectorized_clean_parent_comment',
       'cosine_similarity_dirty_comments', 'cosine_similarity_clean_comments',
       'word_count', 'punctuation_count', 'has_repeated', 'exclaim_count',
       'qns_mark_count', 'ellipses_mark_count', 'interjection_count',
       'laughter_words_count', 'capitalized_word_count',
       'partial_capital_word_count', 'emoticon_count', 'comment_sentiment'],
      dtype='object')

In [ ]:
scaler = MinMaxScaler()

In [ ]:
test_data = test_data.reset_index()

test_data_to_normalize = test_data.loc[:,['word_count','punctuation_count','has_repeated','exclaim_count','qns_mark_count','ellipses_mark_count','interjection_count','laughter_words_count','capitalized_word_count','partial_capital_word_count','emoticon_count','cosine_similarity_dirty_comments','cosine_similarity_clean_comments']]
norm_test_data_arr = scaler.fit_transform(test_data_to_normalize)
test_norm_df = pd.DataFrame(norm_test_data_arr)

test_norm_df.columns = ['word_count','punctuation_count','has_repeated','exclaim_count','qns_mark_count','ellipses_mark_count','interjection_count','laughter_words_count','capitalized_word_count','partial_capital_word_count','emoticon_count','cosine_similarity_dirty_comments','cosine_similarity_clean_comments']

test_data.word_count = test_norm_df.word_count
test_data.punctuation_count = test_norm_df.punctuation_count
test_data.has_repeated = test_norm_df.has_repeated
test_data.exclaim_count = test_norm_df.exclaim_count
test_data.qns_mark_count = test_norm_df.qns_mark_count
test_data.ellipses_mark_count = test_norm_df.ellipses_mark_count
test_data.interjection_count = test_norm_df.interjection_count
test_data.laughter_words_count = test_norm_df.laughter_words_count
test_data.capitalized_word_count = test_norm_df.capitalized_word_count
test_data.partial_capital_word_count = test_norm_df.partial_capital_word_count
test_data.emoticon_count = test_norm_df.emoticon_count
test_data.cosine_similarity_dirty_comments = test_norm_df.cosine_similarity_dirty_comments
test_data.cosine_similarity_clean_comments = test_norm_df.cosine_similarity_clean_comments

In [ ]:
test_data.isnull().sum()

index                               0
label                               0
comment                             0
author                              0
subreddit                           0
score                               0
ups                                 0
downs                               0
date                                0
created_utc                         0
parent_comment                      0
lemmatized_comment                  0
lemmatized_parent_comment           0
clean_comment                       0
clean_parent_comment                0
lemmatized_clean_comment            0
lemmatized_clean_parent_comment     0
vectorized_comment                  0
vectorized_clean_comment            0
vectorized_parent_comment           0
vectorized_clean_parent_comment     0
cosine_similarity_dirty_comments    0
cosine_similarity_clean_comments    0
word_count                          0
punctuation_count                   0
has_repeated                        0
exclaim_coun

In [ ]:
test_data.to_pickle("./norm_test_data")